In [4]:
import_name = 'importC'

In [9]:
import os, sys, logging
from dotenv import load_dotenv
import json
import pandas as pd
import requests

HTTP_SCHEMA = "https://"
BASEURL = "ns.us.everynet.io"
BASEPATH = "/api/"
API_VERSION = "v1.0"
REQ_URL = HTTP_SCHEMA + BASEURL + BASEPATH + API_VERSION

load_dotenv()
NS_TOKEN = os.getenv('NS_TOKEN')

HEADERS = {"accept": "application/json", "Content-Type": "application/json"}

path = "/devices"
url = REQ_URL + path

# get the file name from the script argument
files_base = "imports/" + import_name

devices_file = files_base + "-devices.csv"
commons_file = files_base + "-common.csv"
logging_file = files_base + "-logging.txt" 
print(F'Devices Filename: {devices_file}')
print(F'Commons Filename: {commons_file}')
print(F'Logging Filename: {logging_file}')

devices_df = pd.read_csv(devices_file, dtype=str)
commons_df = pd.read_csv(commons_file, dtype=str)
print(devices_df)

Devices Filename: imports/importC-devices.csv
Commons Filename: imports/importC-common.csv
Logging Filename: imports/importC-logging.txt
            dev_eui           app_eui  dev_addr              tags
0  000DB531137B353A  0000000000010203  137B353A  LT1-353A C3F8+CW
1  000DB531137C356A  0000000000010203  137C356A  LT1-356A C3F9+G5
2  000DB53113633558  0000000000010203  13633558  LT1-3558 C3F9+96


In [6]:
# Create the full import data frame
deviceimport_df = devices_df

# Add "commons" columns if not present in devices file
if 'app_eui' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(app_eui = str(commons_df['app_eui'][0]))
if 'activation' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(activation = commons_df['activation'][0])
if 'appskey' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(appskey = str(commons_df['appskey'][0]))
if 'nwkskey' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(nwkskey = str(commons_df['nwkskey'][0]))
if 'encryption' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(encryption = commons_df['encryption'][0])
if 'dev_class' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(dev_class = commons_df['dev_class'][0])
if 'counters_size' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(counters_size = commons_df['counters_size'][0])
if 'adr' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(adr = commons_df['adr'][0])
if 'tx_power' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(tx_power = commons_df['tx_power'][0])
if 'datarate' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(datarate = commons_df['datarate'][0])
if 'band' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(band = commons_df['band'][0])
if 'common_tags' not in devices_df.columns:
  deviceimport_df = deviceimport_df.assign(common_tags = commons_df['common_tags'][0])
  
print(deviceimport_df)


            dev_eui           app_eui  dev_addr              tags activation   
0  000DB531137B353A  0000000000010203  137B353A  LT1-353A C3F8+CW        abp  \
1  000DB531137C356A  0000000000010203  137C356A  LT1-356A C3F9+G5        abp   
2  000DB53113633558  0000000000010203  13633558  LT1-3558 C3F9+96        abp   

                            appskey                           nwkskey   
0  1628ae2b7e15d2a6abf7cf4f3c158809  28aed22b7e1516a609cfabf715884f3c  \
1  1628ae2b7e15d2a6abf7cf4f3c158809  28aed22b7e1516a609cfabf715884f3c   
2  1628ae2b7e15d2a6abf7cf4f3c158809  28aed22b7e1516a609cfabf715884f3c   

  encryption dev_class counters_size adr  tx_power  datarate       band   
0         ns         a             4  on       NaN       NaN  us902-928  \
1         ns         a             4  on       NaN       NaN  us902-928   
2         ns         a             4  on       NaN       NaN  us902-928   

           common_tags  
0  kris bulk-e importC  
1  kris bulk-e importC  
2  kris bu

In [10]:
# Process the import dataframe, create device (post) at NS
for i in range(0, len(deviceimport_df)):
  # get the row
  row=deviceimport_df.iloc[i]
  data = {}

  # prep the tags
  tags=row['tags'].split(' ')
  common_tags=row['common_tags'].split(' ')

  data['dev_eui'] = row['dev_eui'].lower()
  data['app_eui'] = row['app_eui']
  print(data['app_eui'])
  data['activation'] = row['activation'].upper()
  if data['activation'] == 'OTAA':
    data['app_key'] = row['app_key']
  elif data['activation'] == 'ABP':
    data['dev_addr'] = row['dev_addr']
    data['appskey'] = row['appskey']
    data['nwkskey'] = row['nwkskey']
  data['encryption'] = row['encryption'].upper()
  data['dev_class'] = row['dev_class'].upper()
  data['counters_size'] = int(row['counters_size'])
  data['band'] = row['band'].upper()

  adr = {}
  adr['mode'] = row['adr']
  # adr['tx_power'] = row['tx_power']
  # adr['datarate'] = row['datarate']
  data['adr'] = adr

  data['tags'] = tags + common_tags

  data_json = json.dumps(data)
  print(data_json)

  # response = requests.post(url, data_json, params={"access_token": NS_TOKEN}, headers=HEADERS)
  # if response.status_code == 201:
  #   # print(response.request.url)
  #   # print(response.request.headers)
  #   # print(response.request.body)
  #   message = F"Success: Device Added {data['dev_eui']}, Status Code: {response.status_code}, reason:{response.reason}"
  #   logger.info(message)
  #   print(message)

  # else:
  #   # print("--------------------\nRequest Details:")
  #   # print(response.request.url)
  #   # print(response.request.headers)
  #   # print(response.request.body)
  #   message = F"Failed: Device NOT ADDED: {data['dev_eui']}, Status Code: {response.status_code}, reason:{response.reason}, text:{response.text}"
  #   logger.info(message)  
  #   print(message)

0000000000010203
{"dev_eui": "000db531137b353a", "app_eui": "0000000000010203", "activation": "ABP", "dev_addr": "137B353A", "appskey": "1628ae2b7e15d2a6abf7cf4f3c158809", "nwkskey": "28aed22b7e1516a609cfabf715884f3c", "encryption": "NS", "dev_class": "A", "counters_size": 4, "band": "US902-928", "adr": {"mode": "on"}, "tags": ["LT1-353A", "C3F8+CW", "kris", "bulk-e", "importC"]}
0000000000010203
{"dev_eui": "000db531137c356a", "app_eui": "0000000000010203", "activation": "ABP", "dev_addr": "137C356A", "appskey": "1628ae2b7e15d2a6abf7cf4f3c158809", "nwkskey": "28aed22b7e1516a609cfabf715884f3c", "encryption": "NS", "dev_class": "A", "counters_size": 4, "band": "US902-928", "adr": {"mode": "on"}, "tags": ["LT1-356A", "C3F9+G5", "kris", "bulk-e", "importC"]}
0000000000010203
{"dev_eui": "000db53113633558", "app_eui": "0000000000010203", "activation": "ABP", "dev_addr": "13633558", "appskey": "1628ae2b7e15d2a6abf7cf4f3c158809", "nwkskey": "28aed22b7e1516a609cfabf715884f3c", "encryption": "